In [47]:
import re
import shutil
import json
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('..')

from calcimetry.mongo_api import MongoInfo, MongoAPI

In [2]:
mongo_info = MongoInfo()

# Check number of documents per image

In [18]:
imagedf = pd.read_csv('/work/armitagj/data/csvs/Rapport_Img_Disk2.csv', sep=';')
imagedf

,N,Dossiers,DIR,.jpg,i0,i1,Shift / Miss,Click Locally,ORDER,CHAIN_RULE,Unnamed: 10
0,1.0,BPE4023,Photos,8.0,0,7,NaN,NaN,1.0,VRAI,NaN
1,2.0,BPE4024,Photos,8.0,8,15,NaN,NaN,2.0,FAUX,NaN
2,3.0,DCN2001,Photos,16.0,2946,2961,Shift 2967/2982,NaN,153.0,FAUX,NaN
3,4.0,DCN2002,Photos,4.0,16,19,NaN,NaN,3.0,VRAI,NaN
4,5.0,DCN2003,Photos,15.0,20,34,NaN,NaN,4.0,VRAI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
153,154.0,TER1302,Photos,21.0,2782,2802,Y,TODO,144.0,VRAI,NaN
154,155.0,TPV1321,Photos,65.0,2803,2867,Y,TODO,145.0,VRAI,NaN
155,156.0,TPV1322,Photos,78.0,2868,2945,Y,TODO,146.0,FAUX,NaN
156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#np.where(imagedf.Dossiers == 'BPE4023', imagedf['.jpg'], 0)
imagedf.loc[imagedf['Dossiers'] == 'BPE4023', '.jpg'].iloc[0]

8.0

In [52]:
numdocs = []
docname = []
numorig = []
with MongoAPI(mongo_info=mongo_info) as mongo_api:
    for drillname in imagedf.Dossiers:
        if type(drillname) == str:
            partialdrillname = f'.*{drillname}.*'
            numdocs.append(mongo_api.db['images'].count_documents({'DrillName': {'$regex' : partialdrillname}}))
            docname.append(drillname)
            numorig.append(int(imagedf.loc[imagedf['Dossiers'] == drillname, '.jpg'].iloc[0]))

In [53]:
df = pd.DataFrame(np.array([docname, numorig, numdocs]).T, columns=['DrillName', 'Images on local', 'Images digitized'])
df

,DrillName,Images on local,Images digitized
0,BPE4023,8,8
1,BPE4024,8,8
2,DCN2001,16,16
3,DCN2002,4,4
4,DCN2003,15,15
...,...,...,...
151,TER1202,22,22
152,TER1203,19,19
153,TER1302,21,21
154,TPV1321,65,65


In [54]:
np.where(df['Images digitized'] != df['Images on local'], df['DrillName'], 0) 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 'REP2209', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'SUG2020', 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=object)

In [55]:
np.where(df['Images digitized'] != df['Images on local'], df['Images digitized'], False)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, '0', False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, F